# 검색 API를 사용한 질문 답변 및 순위 재지정

관련 정보를 검색하는 것은 때때로 건초 더미에서 바늘을 찾는 것처럼 느껴질 수 있지만, 절망하지 마세요. 실제로 GPT가 이러한 작업을 상당 부분 대신 수행할 수 있습니다. 이 가이드에서는 다양한 AI 기술로 기존 검색 시스템을 보강하여 노이즈를 걸러내는 데 도움이 되는 방법을 살펴봅니다.

GPT로 정보를 검색하는 두 가지 방법이 있습니다:

1. **인간 브라우징 모방: [GPT가 검색을 트리거하고](https://openai.com/blog/chatgpt-plugins#browsing) 결과를 평가한 후 필요한 경우 검색 쿼리를 수정합니다. 또한 특정 검색 결과를 추적하여 인간 사용자처럼 생각의 사슬을 형성할 수도 있습니다.
2. **임베딩을 사용한 검색: 콘텐츠와 사용자 쿼리에 대한 [임베딩](https://platform.openai.com/docs/guides/embeddings)을 계산한 다음, 코사인 유사도로 측정한 결과 가장 관련성이 높은 [콘텐츠](Question_answering_using_embeddings.ipynb)를 [검색]합니다. 이 기법은 Google과 같은 검색 엔진에서 [많이 사용](https://blog.google/products/search/search-language-understanding-bert/)됩니다.

이 두 가지 접근 방식은 모두 유망하지만 각각 단점이 있습니다. 첫 번째 접근 방식은 반복적인 특성으로 인해 속도가 느릴 수 있고, 두 번째 접근 방식은 전체 지식창고를 미리 임베딩하고 새로운 콘텐츠를 지속적으로 임베딩하며 벡터 데이터베이스를 유지 관리해야 한다는 단점이 있습니다.

이러한 접근 방식을 결합하고 [리랭킹](https://www.sbert.net/examples/applications/retrieve_rerank/README.html) 방법에서 영감을 얻어 그 중간에 위치한 접근 방식을 찾아냈습니다. **이 접근 방식은 Slack 검색 API와 같은 기존 검색 시스템 또는 비공개 데이터가 있는 내부 ElasticSearch 인스턴스** 위에 구현할 수 있습니다. 작동 방식은 다음과 같습니다:

![search_augmented_by_query_생성_및_임베딩_재랭크.png](../images/search_rank_answer.png)

**1단계: 검색**

1.  사용자가 질문을 합니다.
2.  GPT가 잠재적 쿼리 목록을 생성합니다.
3.  3. 검색 쿼리가 병렬로 실행됩니다.

**2단계: 순위 재조정**

1.  각 결과에 대한 임베딩을 사용하여 사용자 질문에 대해 생성된 가상의 이상적인 답변과 의미적 유사성을 계산합니다.
2.  이 유사성 지표를 기준으로 결과의 순위가 매겨지고 필터링됩니다.

**3단계: 답변**

1.  상위 검색 결과가 주어지면 모델은 참조 및 링크를 포함하여 사용자의 질문에 대한 답변을 생성합니다.

이 하이브리드 접근 방식은 상대적으로 짧은 지연 시간을 제공하며 벡터 데이터베이스를 유지 관리할 필요 없이 기존의 모든 검색 엔드포인트에 통합할 수 있습니다. 자세히 살펴보겠습니다! 검색할 도메인으로 [뉴스 API](https://newsapi.org/)를 예시로 사용하겠습니다.

## 설정

사용자 환경에는 `OPENAI_API_KEY` 외에 `NEWS_API_KEY`를 포함해야 합니다. API 키는 [여기](https://newsapi.org/)에서 얻을 수 있습니다.


In [1]:
%%capture
%env NEWS_API_KEY = YOUR_NEWS_API_KEY


In [2]:
# Dependencies
from datetime import date, timedelta  # date handling for fetching recent news
from IPython import display  # for pretty printing
import json  # for parsing the JSON api responses and model outputs
from numpy import dot  # for cosine similarity
import openai  # for using GPT and getting embeddings
import os  # for loading environment variables
import requests  # for making the API requests
from tqdm.notebook import tqdm  # for printing progress bars

# Load environment variables
news_api_key = os.getenv("NEWS_API_KEY")

GPT_MODEL = "gpt-3.5-turbo"


# Helper functions
def json_gpt(input: str):
    completion = openai.ChatCompletion.create(
        model=GPT_MODEL,
        messages=[
            {"role": "system", "content": "Output only valid JSON"},
            {"role": "user", "content": input},
        ],
        temperature=0.5,
    )

    text = completion.choices[0].message.content
    parsed = json.loads(text)

    return parsed


def embeddings(input: list[str]) -> list[list[str]]:
    response = openai.Embedding.create(model="text-embedding-ada-002", input=input)
    return [data.embedding for data in response.data]

## 1. 검색

모든 것은 사용자 질문에서 시작됩니다.


In [3]:
# User asks a question
USER_QUESTION = "Who won the NBA championship? And who was the MVP? Tell me a bit about the last game."

이제 최대한 철저하게 하기 위해 모델을 사용하여 이 질문을 기반으로 다양한 쿼리 목록을 생성합니다.


In [4]:
QUERIES_INPUT = f"""
You have access to a search API that returns recent news articles.
Generate an array of search queries that are relevant to this question.
Use a variation of related keywords for the queries, trying to be as general as possible.
Include as many queries as you can think of, including and excluding terms.
For example, include queries like ['keyword_1 keyword_2', 'keyword_1', 'keyword_2'].
Be creative. The more queries you include, the more likely you are to find relevant results.

User question: {USER_QUESTION}

Format: {{"queries": ["query_1", "query_2", "query_3"]}}
"""

queries = json_gpt(QUERIES_INPUT)["queries"]

# Let's include the original question as well for good measure
queries.append(USER_QUESTION)

queries

['NBA championship winner',
 'MVP of NBA championship',
 'Last game of NBA championship',
 'NBA finals winner',
 'Most valuable player of NBA championship',
 'Finals game of NBA',
 'Who won the NBA finals',
 'NBA championship game summary',
 'NBA finals MVP',
 'Champion of NBA playoffs',
 'NBA finals last game highlights',
 'NBA championship series result',
 'NBA finals game score',
 'NBA finals game recap',
 'NBA champion team and player',
 'NBA finals statistics',
 'NBA championship final score',
 'NBA finals best player',
 'NBA playoffs champion and MVP',
 'NBA finals game analysis',
 'Who won the NBA championship? And who was the MVP? Tell me a bit about the last game.']

쿼리가 좋아 보이므로 검색을 실행해 보겠습니다.


In [5]:
def search_news(
    query: str,
    news_api_key: str = news_api_key,
    num_articles: int = 50,
    from_datetime: str = "2023-06-01",  # the 2023 NBA finals were played in June 2023
    to_datetime: str = "2023-06-30",
) -> dict:
    response = requests.get(
        "https://newsapi.org/v2/everything",
        params={
            "q": query,
            "apiKey": news_api_key,
            "pageSize": num_articles,
            "sortBy": "relevancy",
            "from": from_datetime,
            "to": to_datetime,
        },
    )

    return response.json()


articles = []

for query in tqdm(queries):
    result = search_news(query)
    if result["status"] == "ok":
        articles = articles + result["articles"]
    else:
        raise Exception(result["message"])

# remove duplicates
articles = list({article["url"]: article for article in articles}.values())

print("Total number of articles:", len(articles))
print("Top 5 articles of query 1:", "\n")

for article in articles[0:5]:
    print("Title:", article["title"])
    print("Description:", article["description"])
    print("Content:", article["content"][0:100] + "...")
    print()


  0%|          | 0/21 [00:00<?, ?it/s]

Total number of articles: 554
Top 5 articles of query 1: 

Title: Nascar takes on Le Mans as LeBron James gets centenary race under way
Description: <ul><li>Nascar has presence at iconic race for first time since 1976</li><li>NBA superstar LeBron James waves flag as honorary starter</li></ul>The crowd chanted “U-S-A! U-S-A!” as Nascar driver lineup for the 24 Hours of Le Mans passed through the city cente…
Content: The crowd chanted U-S-A! U-S-A! as Nascar driver lineup for the 24 Hours of Le Mans passed through t...

Title: NBA finals predictions: Nuggets or Heat? Our writers share their picks
Description: Denver or Miami? Our contributors pick the winner, key players and dark horses before the NBA’s grand finale tips offA lot has been made of the importance of a balanced roster with continuity, but, somehow, still not enough. The Nuggets are the prime example …
Content: The Nuggets are here because 
A lot has been made of the importance of a balanced roster with conti...

Title: Unbo

보시다시피, 검색 쿼리는 종종 많은 수의 결과를 반환하며, 이 중 상당수는 사용자가 원래 질문한 내용과 관련이 없는 경우가 많습니다. 최종 답변의 품질을 개선하기 위해 임베딩을 사용하여 결과의 순위를 다시 정하고 필터링합니다.

## 2. 순위 재조정

HyDE(Gao et al.)](https://arxiv.org/abs/2212.10496)에서 영감을 얻어, 먼저 가상의 이상적인 답변을 생성하여 결과 비교의 순위를 다시 매깁니다. 이렇게 하면 질문과 유사한 결과보다는 좋은 답변처럼 보이는 결과의 우선순위를 정하는 데 도움이 됩니다. 다음은 가상의 답변을 생성하는 데 사용하는 프롬프트입니다.


In [6]:
HA_INPUT = f"""
Generate a hypothetical answer to the user's question. This answer will be used to rank search results. 
Pretend you have all the information you need to answer, but don't use any actual facts. Instead, use placeholders
like NAME did something, or NAME said something at PLACE. 

User question: {USER_QUESTION}

Format: {{"hypotheticalAnswer": "hypothetical answer text"}}
"""

hypothetical_answer = json_gpt(HA_INPUT)["hypotheticalAnswer"]

hypothetical_answer


'The NBA championship was won by TEAM NAME. The MVP was awarded to PLAYER NAME. The last game was held at STADIUM NAME, where both teams played with great energy and enthusiasm. It was a close game, but in the end, TEAM NAME emerged victorious.'

이제 검색 결과와 가상의 답변에 대한 임베딩을 생성해 보겠습니다. 그런 다음 이러한 임베딩 간의 코사인 거리를 계산하여 의미적 유사성 지표를 제공합니다. OpenAI 임베딩은 API에서 정규화된 상태로 반환되므로 전체 코사인 유사도 계산을 수행하는 대신 도트 곱을 간단히 계산할 수 있습니다.


In [7]:
hypothetical_answer_embedding = embeddings(hypothetical_answer)[0]
article_embeddings = embeddings(
    [
        f"{article['title']} {article['description']} {article['content'][0:100]}"
        for article in articles
    ]
)

# Calculate cosine similarity
cosine_similarities = []
for article_embedding in article_embeddings:
    cosine_similarities.append(dot(hypothetical_answer_embedding, article_embedding))

cosine_similarities[0:10]


[0.7854456526852069,
 0.8086023500072106,
 0.8002998147018501,
 0.7961229569526956,
 0.798354506673743,
 0.758216458795653,
 0.7753754083127359,
 0.7494958338411927,
 0.804733946801739,
 0.8405965885235218]

마지막으로 이러한 유사성 점수를 사용하여 결과를 정렬하고 필터링합니다.


In [8]:
scored_articles = zip(articles, cosine_similarities)

# Sort articles by cosine similarity
sorted_articles = sorted(scored_articles, key=lambda x: x[1], reverse=True)

# Print top 5 articles
print("Top 5 articles:", "\n")

for article, score in sorted_articles[0:5]:
    print("Title:", article["title"])
    print("Description:", article["description"])
    print("Content:", article["content"][0:100] + "...")
    print("Score:", score)
    print()


Top 5 articles: 

Title: NBA Finals: Denver Nuggets beat Miami Hea, lift thier first-ever NBA title
Description: Denver Nuggets won their maiden NBA Championship trophy defeating Miami Heat 94-89 in Game 5 of the NBA Final held on Tuesday at the Ball Arena in Denver
Content: Denver Nuggets won their maiden NBA Championship trophy defeating Miami Heat 94-89 in Game 5 of the ...
Score: 0.8445817523602124

Title: Photos: Denver Nuggets celebrate their first NBA title
Description: The Nuggets capped off an impressive postseason by beating the Miami Heat in the NBA Finals.
Content: Thousands of supporters watched along the streets of Denver, Colorado as the US National Basketball ...
Score: 0.842070667753606

Title: Denver Nuggets win first NBA championship title in Game 5 victory over Miami Heat
Description: The Denver Nuggets won their first NBA championship Monday night, downing the Miami Heat 94-89 at Ball Arena in Denver to take Game 5 of the NBA Finals.
Content: The Denver Nuggets won

멋지네요! 이 결과는 원래 쿼리와 훨씬 더 관련성이 높아 보입니다. 이제 상위 5개 결과를 사용하여 최종 답변을 생성해 보겠습니다.

## 3. Answer


In [9]:
formatted_top_results = [
    {
        "title": article["title"],
        "description": article["description"],
        "url": article["url"],
    }
    for article, _score in sorted_articles[0:5]
]

ANSWER_INPUT = f"""
Generate an answer to the user's question based on the given search results. 
TOP_RESULTS: {formatted_top_results}
USER_QUESTION: {USER_QUESTION}

Include as much information as possible in the answer. Reference the relevant search result urls as markdown links.
"""

completion = openai.ChatCompletion.create(
    model=GPT_MODEL,
    messages=[{"role": "user", "content": ANSWER_INPUT}],
    temperature=0.5,
    stream=True,
)

text = ""
for chunk in completion:
    text += chunk.choices[0].delta.get("content", "")
    display.clear_output(wait=True)
    display.display(display.Markdown(text))

The Denver Nuggets won their first-ever NBA championship by defeating the Miami Heat 94-89 in Game 5 of the NBA Finals held on Tuesday at the Ball Arena in Denver, according to this [Business Standard article](https://www.business-standard.com/sports/other-sports-news/nba-finals-denver-nuggets-beat-miami-hea-lift-thier-first-ever-nba-title-123061300285_1.html). Nikola Jokic, the Nuggets' center, was named the NBA Finals MVP. In a rock-fight of a Game 5, the Nuggets reached the NBA mountaintop, securing their franchise's first NBA championship and setting Nikola Jokic's legacy as an all-timer in stone, according to this [Yahoo Sports article](https://sports.yahoo.com/nba-finals-nikola-jokic-denver-nuggets-survive-miami-heat-to-secure-franchises-first-nba-championship-030321214.html). For more information and photos of the Nuggets' celebration, check out this [Al Jazeera article](https://www.aljazeera.com/gallery/2023/6/15/photos-denver-nuggets-celebrate-their-first-nba-title) and this [CNN article](https://www.cnn.com/2023/06/12/sport/denver-nuggets-nba-championship-spt-intl?cid=external-feeds_iluminar_yahoo).